In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [ ]:
# Load dataset
df = pd.read_csv('sales_data_sample.csv', encoding='unicode_escape')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# Drop unnecessary columns
to_drop = ['ADDRESSLINE1', 'ADDRESSLINE2', 'STATE', 'POSTALCODE', 'PHONE']
df = df.drop(to_drop, axis=1)

In [ ]:
#Check for null values
df.isnull().sum()

In [ ]:
#Bhai bhai look at territory, Vishwajeet shocked other admin rocked
#But territory does not have significant impact on analysis like RFM segmentation or clustering so ignore it

In [ ]:
df.dtypes

In [ ]:
#ORDERDATE Should be in date time
df['ORDERDATE'] = pd.to_datetime(df['ORDERDATE'])

In [ ]:
# RFM Feature Calculation
snapshot_date = df['ORDERDATE'].max() + dt.timedelta(days=1)
df_RFM = df.groupby('CUSTOMERNAME').agg({
    'ORDERDATE': lambda x: (snapshot_date - x.max()).days,
    'ORDERNUMBER': 'count',
    'SALES': 'sum'
})

In [ ]:
# Rename columns
df_RFM.rename(columns={
    'ORDERDATE': 'Recency',
    'ORDERNUMBER': 'Frequency',
    'SALES': 'MonetaryValue'
}, inplace=True)

In [ ]:
# Quartile segmentation
df_RFM['M'] = pd.qcut(df_RFM['MonetaryValue'], q=4, labels=range(1,5))
df_RFM['R'] = pd.qcut(df_RFM['Recency'], q=4, labels=list(range(4,0,-1)))
df_RFM['F'] = pd.qcut(df_RFM['Frequency'], q=4, labels=range(1,5))

In [ ]:
df_RFM.head()

In [ ]:
# RFM Score
df_RFM['RFM_Score'] = df_RFM[['R','F','M']].sum(axis=1)

In [ ]:
# RFM Level
def rfm_level(df):
    if df['RFM_Score'] >= 10:
        return 'High Value Customer'
    elif 6 <= df['RFM_Score'] < 10:
        return 'Mid Value Customer'
    else:
        return 'Low Value Customer'

In [ ]:
df_RFM['RFM_Level'] = df_RFM.apply(rfm_level, axis=1)
print(df_RFM.head())

In [ ]:
# Data Preparation for Clustering
data = df_RFM[['Recency', 'Frequency', 'MonetaryValue']]

# We create levels for our Customers
### RFM Score > 10 : High Value Customers
### RFM Score < 10 and RFM Score >= 6 : Mid Value Customers
### RFM Score < 6 : Low Value Customers


In [ ]:
# Log transformation to reduce skew
data_log = np.log1p(data)  # log(1 + x)

In [ ]:
# Standardization
scaler = StandardScaler()
data_normalized = pd.DataFrame(scaler.fit_transform(data_log),
                               index=data_log.index,
                               columns=data_log.columns)
print(data_normalized.describe().round(2))

In [ ]:
# Our data is skewed we must remove it by performing log transformation
data_log = np.log(data)
data_log.head()

In [ ]:
#Fit KMeans and use elbow method to choose the number of clusters
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans

sse = {}

for k in range(1, 21):
    kmeans = KMeans(n_clusters = k, random_state = 1)
    kmeans.fit(data_normalized)
    sse[k] = kmeans.inertia_


In [ ]:
plt.figure(figsize=(10,6))
plt.title('The Elbow Method')

plt.xlabel('K')
plt.ylabel('SSE')
plt.style.use('ggplot')

sns.pointplot(x=list(sse.keys()), y = list(sse.values()))
plt.text(4.5, 60, "Largest Angle", bbox = dict(facecolor = 'lightgreen', alpha = 0.5))
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=5, random_state=1)
kmeans.fit(data_normalized)
cluster_labels = kmeans.labels_

data_rfm = data.assign(Cluster = cluster_labels)
data_rfm.head()